In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

C:\Users\annan\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
asylum = pd.read_csv('df_finale.csv').drop('Unnamed: 0',axis=1)
immig = pd.read_csv('immig.csv').drop('Unnamed: 0',axis=1)

In [3]:
asylum = asylum[asylum['OBS_VALUE']!=0]
immig = immig[immig['OBS_VALUE']!=0].drop(['sex_y'],axis=1).rename(columns={"sex_x":"sex"})

In [4]:
def indicatrice_sex(string):
    if string == 'M':
        return 1
    else : 
        return 0

asylum['indicatrice_sex'] = asylum['sex'].apply(indicatrice_sex)
immig['indicatrice_sex'] = immig['sex'].apply(indicatrice_sex)

In [5]:
def mettre_sous_forme_liste(s):
    if (s == 1) :
        return ['NA']
    lst = [elem for elem in s.split("'") if elem.strip()]
    lst.remove('[')
    lst.remove(']')
    return lst

In [6]:
asylum['Langues_citizen'] = asylum['Langues_citizen'].apply(mettre_sous_forme_liste)
immig['Langues_citizen'] = immig['Langues_citizen'].apply(mettre_sous_forme_liste)
asylum['Langues_geo'] = asylum['Langues_geo'].apply(mettre_sous_forme_liste)
immig['Langues_geo'] = immig['Langues_geo'].apply(mettre_sous_forme_liste)

In [7]:
asylum['Indicatrice_langue'] = 0
for i in tqdm(asylum.index):
    langue_citizen = asylum['Langues_citizen'][i]
    langue_geo = asylum['Langues_geo'][i]
    for ele in langue_citizen:
        for ele2 in langue_geo:
            if ele == ele2:
                asylum.loc[i,'Indicatrice_langue'] = 1
                
immig['Indicatrice_langue'] = 0
for i in tqdm(immig.index):
    langue_citizen = immig['Langues_citizen'][i]
    langue_geo = immig['Langues_geo'][i]
    for ele in langue_citizen:
        for ele2 in langue_geo:
            if ele == ele2:
                immig.loc[i,'Indicatrice_langue'] = 1

100%|█████████████████████████████████████████████████████████████████████████| 67005/67005 [00:01<00:00, 37242.99it/s]


In [9]:
asylum['OBS_VALUE_POND'] = (asylum['OBS_VALUE'] / asylum['pop_citizen'])*10000000
asylum['gap_idh'] = asylum['idh_geo']/asylum['idh_citizen']
asylum['gap_pib'] = asylum['pib_geo']/asylum['pib_citizen']
asylum['gap_infla'] = asylum['infla_geo']/asylum['infla_citizen']
asylum['gap_unemp'] = asylum['unemp_dest']/asylum['unemp_origine']


immig['OBS_VALUE_POND'] = (immig['OBS_VALUE'] / immig['pop_citizen'])*10000000
immig['gap_idh'] = immig['idh_geo']/immig['idh_citizen']
immig['gap_pib'] = immig['pib_geo']/immig['pib_citizen']
immig['gap_infla'] = immig['infla_geo']/immig['infla_citizen']
immig['gap_unemp'] = immig['unemp_dest']/immig['unemp_origine']


In [10]:
asylum['religion_commune'] = np.where(asylum['religion_commune']== "Oui", 1, 0)
immig['religion_commune'] = np.where(immig['religion_commune']== "Oui", 1, 0)

## Régressions emboîtées pour demandes d'asile

In [22]:
dest = ['OBS_VALUE_POND','pib_geo', 'idh_geo',  'infla_geo',  'pop_geo', 'unemp_dest',
        'fh_ep_dest','fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest', 'dr_ig_dest','ht_region_dest', 'wbgi_pve_dest']

asylum_dest = asylum[dest]

In [23]:
asylum_dest['pib_geo'] = np.log(asylum_dest['pib_geo'])

<ipython-input-23-a2b9317cf12e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asylum_dest['pib_geo'] = np.log(asylum_dest['pib_geo'])


In [24]:
asylum_dest.drop('OBS_VALUE_POND',axis=1).columns

Index(['pib_geo', 'idh_geo', 'infla_geo', 'pop_geo', 'unemp_dest',
       'fh_ep_dest', 'fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest',
       'dr_ig_dest', 'ht_region_dest', 'wbgi_pve_dest'],
      dtype='object')

In [25]:
x = asylum_dest.drop('OBS_VALUE_POND',axis=1)
Y = asylum_dest[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_dest = sm.OLS(Y, x).fit()
print(model_dest.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     133.9
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        10:35:33   Log-Likelihood:            -6.4713e+05
No. Observations:               82667   AIC:                         1.294e+06
Df Residuals:                   82654   BIC:                         1.294e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        239.8866    102.141      2.349      0.0

In [26]:
asylum['ht_region_dest']

13         0.777778
14         0.777778
15         0.777778
17         0.777778
19         0.777778
             ...   
1296832    0.222222
1296833    0.222222
1296976    0.222222
1297012    0.222222
1297156    0.000000
Name: ht_region_dest, Length: 82667, dtype: float64

In [27]:
origine = ['pib_citizen','idh_citizen','infla_citizen','unemp_origine','fh_status_origine', 'fh_feb_origine',
       'wbgi_cce_origine', 'fh_rol_origine', 'fh_cl_origine',
       'fh_pair_origine', 'wbgi_rle_origine', 'wbgi_pve_origine','indicatrice_sex']

dest_origine = dest+origine

asylum_dest_origine = asylum[dest_origine]

In [28]:
asylum_dest_origine['pib_geo'] = np.log(asylum_dest_origine['pib_geo'])
asylum_dest_origine['pib_citizen'] = np.log(asylum_dest_origine['pib_citizen'])

<ipython-input-28-2827e1a8cf70>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asylum_dest_origine['pib_geo'] = np.log(asylum_dest_origine['pib_geo'])
<ipython-input-28-2827e1a8cf70>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asylum_dest_origine['pib_citizen'] = np.log(asylum_dest_origine['pib_citizen'])


In [29]:
x = asylum_dest_origine.drop('OBS_VALUE_POND',axis=1)
Y = asylum_dest_origine[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_dest_origine = sm.OLS(Y, x).fit()
print(model_dest_origine.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     109.1
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        10:35:34   Log-Likelihood:            -6.4663e+05
No. Observations:               82667   AIC:                         1.293e+06
Df Residuals:                   82642   BIC:                         1.294e+06
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        598.5369    113.100      5.292      0.0

In [30]:
bi = ['religion_commune','Indicatrice_langue','dist_capit']

bi_dest_origine = dest + origine + bi

asylum_bi_dest_origine = asylum[bi_dest_origine]

In [31]:
asylum_bi_dest_origine['pib_citizen'] = np.log(asylum_bi_dest_origine['pib_citizen'])
asylum_bi_dest_origine['dist_capit_carré'] = np.square(asylum_bi_dest_origine['dist_capit'])
asylum_bi_dest_origine['pib_geo'] = np.log(asylum_bi_dest_origine['pib_geo'])

<ipython-input-31-45a8f5d42959>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asylum_bi_dest_origine['pib_citizen'] = np.log(asylum_bi_dest_origine['pib_citizen'])
<ipython-input-31-45a8f5d42959>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asylum_bi_dest_origine['dist_capit_carré'] = np.square(asylum_bi_dest_origine['dist_capit'])
<ipython-input-31-45a8f5d42959>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [32]:
asylum_bi_dest_origine

,OBS_VALUE_POND,pib_geo,idh_geo,infla_geo,pop_geo,unemp_dest,fh_ep_dest,fh_pr_dest,ipu_l_sw_dest,fh_fog_dest,dr_ig_dest,ht_region_dest,wbgi_pve_dest,pib_citizen,idh_citizen,infla_citizen,unemp_origine,fh_status_origine,fh_feb_origine,wbgi_cce_origine,fh_rol_origine,fh_cl_origine,fh_pair_origine,wbgi_rle_origine,wbgi_pve_origine,indicatrice_sex,religion_commune,Indicatrice_langue,dist_capit,dist_capit_carré
13,11.351941,10.881175,0.898,1.956322,8321496.0,6.18,0.500000,0.666667,0.434169,0.250000,0.129399,0.777778,0.125517,7.350416,0.430,2.096289,11.71,1.0,0.3125,0.041230,0.1250,0.833333,0.3125,0.145806,0.125517,0,0,0,12723.326353,1.618830e+08
14,62.435675,10.881175,0.898,1.956322,8321496.0,6.18,0.500000,0.666667,0.434169,0.250000,0.129399,0.777778,0.125517,7.350416,0.430,2.096289,11.71,1.0,0.3125,0.041230,0.1250,0.833333,0.3125,0.145806,0.125517,0,0,0,12723.326353,1.618830e+08
15,18.919901,10.881175,0.898,1.956322,8321496.0,6.18,0.500000,0.666667,0.434169,0.250000,0.129399,0.777778,0.125517,7.350416,0.430,2.096289,11.71,1.0,0.3125,0.041230,0.1250,0.833333,0.3125,0.145806,0.125517,0,0,0,12723.326353,1.618830e+08
17,62.435675,10.881175,0.898,1.956322,8321496.0,6.18,0.500000,0.666667,0.434169,0.250000,0.129399,0.777778,0.125517,7.350416,0.430,2.096289,11.71,1.0,0.3125,0.041230,0.1250,0.833333,0.3125,0.145806,0.125517,0,0,0,12723.326353,1.618830e+08
19,109.735428,10.881175,0.898,1.956322,8321496.0,6.18,0.500000,0.666667,0.434169,0.250000,0.129399,0.777778,0.125517,7.350416,0.430,2.096289,11.71,1.0,0.3125,0.041230,0.1250,0.833333,0.3125,0.145806,0.125517,1,0,0,12723.326353,1.618830e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296832,4.689471,10.369296,0.848,2.383314,5447247.0,9.51,0.000000,1.000000,0.175549,0.000000,0.323911,0.222222,0.131154,6.279244,0.577,54.603848,10.94,1.0,0.1875,0.015595,0.0000,1.000000,0.0625,0.110428,0.131154,1,0,0,13321.926162,1.774737e+08
1296833,2.344735,10.369296,0.848,2.383314,5447247.0,9.51,0.000000,1.000000,0.175549,0.000000,0.323911,0.222222,0.131154,6.279244,0.577,54.603848,10.94,1.0,0.1875,0.015595,0.0000,1.000000,0.0625,0.110428,0.131154,1,0,0,13321.926162,1.774737e+08
1296976,8.154647,10.369296,0.848,2.383314,5447247.0,9.51,0.750000,0.333333,0.412226,0.333333,0.417715,0.222222,0.528487,9.249363,0.731,4.536053,12.44,0.5,0.7500,0.377939,0.5625,0.333333,0.6250,0.569700,0.528487,1,0,0,13377.262009,1.789511e+08
1297012,1.179587,10.369296,0.848,2.383314,5447247.0,9.51,0.416667,0.666667,0.271160,0.250000,0.683076,0.222222,0.447242,10.356681,0.838,28.950586,9.71,1.0,0.3125,0.341433,0.1875,0.833333,0.3125,0.461101,0.447242,1,0,0,19321.349491,3.733145e+08


In [33]:
x = asylum_bi_dest_origine.drop('OBS_VALUE_POND',axis=1)
Y = asylum_bi_dest_origine[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_bi_dest_origine = sm.OLS(Y, x).fit()
print(model_bi_dest_origine.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     94.59
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        10:35:38   Log-Likelihood:            -6.4662e+05
No. Observations:               82667   AIC:                         1.293e+06
Df Residuals:                   82638   BIC:                         1.294e+06
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        572.0190    113.841      5.025      0.0

In [34]:
asylum_bi_dest_origine.columns

Index(['OBS_VALUE_POND', 'pib_geo', 'idh_geo', 'infla_geo', 'pop_geo',
       'unemp_dest', 'fh_ep_dest', 'fh_pr_dest', 'ipu_l_sw_dest',
       'fh_fog_dest', 'dr_ig_dest', 'ht_region_dest', 'wbgi_pve_dest',
       'pib_citizen', 'idh_citizen', 'infla_citizen', 'unemp_origine',
       'fh_status_origine', 'fh_feb_origine', 'wbgi_cce_origine',
       'fh_rol_origine', 'fh_cl_origine', 'fh_pair_origine',
       'wbgi_rle_origine', 'wbgi_pve_origine', 'indicatrice_sex',
       'religion_commune', 'Indicatrice_langue', 'dist_capit',
       'dist_capit_carré'],
      dtype='object')

### Etude avec les ratios macro

In [67]:
dest = ['OBS_VALUE_POND', 'pib_geo', 'idh_geo',  'infla_geo',  'pop_geo', 'unemp_dest',
        'fh_ep_dest','fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest', 'dr_ig_dest','ht_region_dest', 'wbgi_pve_dest']

asylum_dest = asylum[dest]

x = asylum_dest.drop('OBS_VALUE_POND',axis=1)
Y = asylum_dest[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_origine = sm.OLS(Y, x).fit()
print(model_origine.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     133.5
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        18:33:04   Log-Likelihood:            -6.4713e+05
No. Observations:               82667   AIC:                         1.294e+06
Df Residuals:                   82654   BIC:                         1.294e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.6573     89.105      0.322      0.7

In [63]:
origine = ['pib_citizen','idh_citizen','infla_citizen','unemp_origine','fh_status_origine', 'fh_feb_origine',
       'wbgi_cce_origine', 'fh_rol_origine', 'fh_cl_origine',
       'fh_pair_origine', 'wbgi_rle_origine', 'wbgi_pve_origine','indicatrice_sex']

dest_origine = dest+origine

asylum_dest_origine = asylum[dest_origine]

x = asylum_dest_origine.drop('OBS_VALUE_POND',axis=1)
Y = asylum_dest_origine[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_dest_origine = sm.OLS(Y, x).fit()
print(model_dest_origine.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     99.69
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        18:18:39   Log-Likelihood:            -6.4674e+05
No. Observations:               82667   AIC:                         1.294e+06
Df Residuals:                   82642   BIC:                         1.294e+06
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -52.9900    102.537     -0.517      0.6

In [64]:
bi = ['religion_commune','Indicatrice_langue','gap_idh', 'gap_pib','gap_infla', 'gap_unemp','dist_capit']

bi_dest_origine = dest_origine + bi

asylum_bi_dest_origine = asylum[bi_dest_origine]

asylum_bi_dest_origine['dist_capit_carré'] = np.square(asylum_bi_dest_origine['dist_capit'])

x = asylum_bi_dest_origine.drop('OBS_VALUE_POND',axis=1)
Y = asylum_bi_dest_origine[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_asylum_bi_dest_origine = sm.OLS(Y, x).fit()
print(model_asylum_bi_dest_origine.summary())

<ipython-input-64-07dee7ad1972>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asylum_bi_dest_origine['dist_capit_carré'] = np.square(asylum_bi_dest_origine['dist_capit'])


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     79.28
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        18:18:51   Log-Likelihood:            -6.4667e+05
No. Observations:               82667   AIC:                         1.293e+06
Df Residuals:                   82634   BIC:                         1.294e+06
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -578.1974    129.808     -4.454      0.0

In [65]:
asylum_bi_dest_origine.drop('OBS_VALUE_POND',axis=1).columns

Index(['pib_geo', 'idh_geo', 'infla_geo', 'pop_geo', 'unemp_dest',
       'fh_ep_dest', 'fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest',
       'dr_ig_dest', 'ht_region_dest', 'wbgi_pve_dest', 'pib_citizen',
       'idh_citizen', 'infla_citizen', 'unemp_origine', 'fh_status_origine',
       'fh_feb_origine', 'wbgi_cce_origine', 'fh_rol_origine', 'fh_cl_origine',
       'fh_pair_origine', 'wbgi_rle_origine', 'wbgi_pve_origine',
       'indicatrice_sex', 'religion_commune', 'Indicatrice_langue', 'gap_idh',
       'gap_pib', 'gap_infla', 'gap_unemp', 'dist_capit', 'dist_capit_carré'],
      dtype='object')

## Régressions emboîtées pour immigration

In [35]:
dest = ['OBS_VALUE_POND','pib_geo', 'idh_geo',  'infla_geo',  'pop_geo', 'unemp_dest',
        'fh_ep_dest','fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest', 'dr_ig_dest','ht_region_dest', 'wbgi_pve_dest']

immig_dest = immig[dest]

In [36]:
immig_dest['pib_geo'] = np.log(immig_dest['pib_geo'])

<ipython-input-36-7a6ba52bf55c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immig_dest['pib_geo'] = np.log(immig_dest['pib_geo'])


In [37]:
x = immig_dest.drop('OBS_VALUE_POND',axis=1)
Y = immig_dest[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_origine = sm.OLS(Y, x).fit()
print(model_origine.summary())

# Il faut retirer infla_citizen, fh_cl_origine, wbgi_rle_origine

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     169.6
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        10:46:16   Log-Likelihood:            -6.0054e+05
No. Observations:               67005   AIC:                         1.201e+06
Df Residuals:                   66992   BIC:                         1.201e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1829.3878    264.915     -6.906      0.0

In [38]:
origine = ['pib_citizen','idh_citizen','infla_citizen','unemp_origine','fh_status_origine', 'fh_feb_origine',
       'wbgi_cce_origine', 'fh_rol_origine', 'fh_cl_origine',
       'fh_pair_origine', 'wbgi_rle_origine', 'wbgi_pve_origine','indicatrice_sex']

dest_origine = dest+origine

immig_dest_origine = immig[dest_origine]

immig_dest_origine['pib_geo'] = np.log(immig_dest_origine['pib_geo'])
immig_dest_origine['pib_citizen'] = np.log(immig_dest_origine['pib_citizen'])

<ipython-input-38-d747823445f4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immig_dest_origine['pib_geo'] = np.log(immig_dest_origine['pib_geo'])
<ipython-input-38-d747823445f4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immig_dest_origine['pib_citizen'] = np.log(immig_dest_origine['pib_citizen'])


In [39]:
x = immig_dest_origine.drop('OBS_VALUE_POND',axis=1)
Y = immig_dest_origine[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_dest_origine = sm.OLS(Y, x).fit()
print(model_dest_origine.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     92.83
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        10:46:17   Log-Likelihood:            -6.0044e+05
No. Observations:               67005   AIC:                         1.201e+06
Df Residuals:                   66980   BIC:                         1.201e+06
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2043.2969    322.482     -6.336      0.0

In [40]:
bi = ['religion_commune','Indicatrice_langue','dist_capit']

bi_dest_origine = dest_origine + bi

immig_bi_dest_origine = immig[bi_dest_origine]

immig_bi_dest_origine['pib_geo'] = np.log(immig_dest_origine['pib_geo'])
immig_bi_dest_origine['pib_citizen'] = np.log(immig_dest_origine['pib_citizen'])
immig_bi_dest_origine['dist_capit_carré'] = np.square(immig_bi_dest_origine['dist_capit'])

<ipython-input-40-84f5180e8a41>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immig_bi_dest_origine['pib_geo'] = np.log(immig_dest_origine['pib_geo'])
<ipython-input-40-84f5180e8a41>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immig_bi_dest_origine['pib_citizen'] = np.log(immig_dest_origine['pib_citizen'])
<ipython-input-40-84f5180e8a41>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [41]:
x = immig_bi_dest_origine.drop('OBS_VALUE_POND',axis=1)
Y = immig_bi_dest_origine[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_bi_dest_origine = sm.OLS(Y, x).fit()
print(model_bi_dest_origine.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     266.2
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        10:46:19   Log-Likelihood:            -5.9801e+05
No. Observations:               67005   AIC:                         1.196e+06
Df Residuals:                   66976   BIC:                         1.196e+06
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5754.6512    738.747      7.790      0.0

In [42]:
immig_bi_dest_origine.drop('OBS_VALUE_POND',axis=1).columns

Index(['pib_geo', 'idh_geo', 'infla_geo', 'pop_geo', 'unemp_dest',
       'fh_ep_dest', 'fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest',
       'dr_ig_dest', 'ht_region_dest', 'wbgi_pve_dest', 'pib_citizen',
       'idh_citizen', 'infla_citizen', 'unemp_origine', 'fh_status_origine',
       'fh_feb_origine', 'wbgi_cce_origine', 'fh_rol_origine', 'fh_cl_origine',
       'fh_pair_origine', 'wbgi_rle_origine', 'wbgi_pve_origine',
       'indicatrice_sex', 'religion_commune', 'Indicatrice_langue',
       'dist_capit', 'dist_capit_carré'],
      dtype='object')

### Etude avec les ratios macro

In [56]:
dest = ['OBS_VALUE_POND','pop_geo', 'pib_geo', 'idh_geo',  'infla_geo',  'pop_geo', 'unemp_dest',
        'fh_ep_dest','fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest', 'dr_ig_dest','ht_region_dest', 'wbgi_pve_dest']

immig_dest = immig[dest]

x = immig_dest.drop('OBS_VALUE_POND',axis=1)
Y = immig_dest[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_origine = sm.OLS(Y, x).fit()
print(model_origine.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     169.4
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        10:58:44   Log-Likelihood:            -6.0054e+05
No. Observations:               67005   AIC:                         1.201e+06
Df Residuals:                   66992   BIC:                         1.201e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1341.2189    247.304     -5.423      0.0

In [57]:
origine = ['pib_citizen','idh_citizen','infla_citizen','unemp_origine','fh_status_origine', 'fh_feb_origine',
       'wbgi_cce_origine', 'fh_rol_origine', 'fh_cl_origine',
       'fh_pair_origine', 'wbgi_rle_origine', 'wbgi_pve_origine','indicatrice_sex']

dest_origine = dest+origine

immig_dest_origine = immig[dest_origine]

In [58]:
x = immig_dest_origine.drop('OBS_VALUE_POND',axis=1)
Y = immig_dest_origine[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_dest_origine = sm.OLS(Y, x).fit()
print(model_dest_origine.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     93.08
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        10:58:46   Log-Likelihood:            -6.0044e+05
No. Observations:               67005   AIC:                         1.201e+06
Df Residuals:                   66980   BIC:                         1.201e+06
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1636.5977    302.183     -5.416      0.0

In [59]:
bi = ['religion_commune','Indicatrice_langue','gap_idh', 'gap_pib','gap_infla', 'gap_unemp','dist_capit']

bi_dest_origine = dest_origine + bi

immig_bi_dest_origine = immig[bi_dest_origine]

immig_bi_dest_origine['dist_capit_carré'] = np.square(immig_bi_dest_origine['dist_capit'])

<ipython-input-59-33944d555680>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immig_bi_dest_origine['dist_capit_carré'] = np.square(immig_bi_dest_origine['dist_capit'])


In [60]:
x = immig_bi_dest_origine.drop('OBS_VALUE_POND',axis=1)
Y = immig_bi_dest_origine[['OBS_VALUE_POND']]

x = sm.add_constant(x)

x, Y = np.array(x), np.array(Y)

model_bi_dest_origine = sm.OLS(Y, x).fit()
print(model_bi_dest_origine.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     235.4
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        10:58:48   Log-Likelihood:            -5.9797e+05
No. Observations:               67005   AIC:                         1.196e+06
Df Residuals:                   66972   BIC:                         1.196e+06
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        845.2968    377.130      2.241      0.0